In [1]:
!pip install transformers
!pip install torch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 63.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 22.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [13]:
from transformers import RobertaTokenizer, RobertaForCausalLM
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel, BertForSequenceClassification, TrainingArguments, Trainer
import os
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [17]:
current_dir = os.getcwd()
os.chdir("/content/data")
all_files = os.listdir(os.curdir)
all_files.pop(1) # deleting .ipynb_checkpoints
all_files


['1679821796.614989', '1679824559.317385', '1679783106.1030889']

In [18]:
dfs = []
for file in all_files:
  dfs.append(pd.read_csv(file, sep=";", names=["lemma", "correct"]))

os.chdir("/content/")
df = pd.concat(dfs)
df

,lemma,correct
0,<pad> <pad> 46 lastbiler leverer sand til mest...,sand til mesterskab 05 <pad> <pad>
1,kl 16 25 billund varme op til de danske,kl 16 25 billund varmer op til de danske
2,mesterskaber i beachvolley der spille også hån...,mesterskaber i beachvolley der spilles også hå...
3,badestrande men i den komme weekend er byen vært,badestrande men i den kommende weekend er byen...
4,"i beachvolley - vi hørte, at nyborg kommune har","i beachvolley - vi hørte, at nyborg kommune har"
...,...,...
47651,"vasken, så kan de ansætte dog i et vist","vasken, så kan de ansatte dog i et vist"
47652,"i et vist omfang trøste sig med, at der","i et vist omfang trøste sig med, at der"
47653,"der næsten ingen arbejsløshed være i området, ...","der næsten ingen arbejsløshed er i området, så..."
47654,"i området, så der være gode udsigter for komme","i området, så der er gode udsigter for komme"


In [19]:
tokenizer = RobertaTokenizer.from_pretrained("DDSC/roberta-base-danish")

In [20]:
X = list(df["lemma"])
y = list(df["correct"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=1212)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True)
Y_train_tokenized = tokenizer(y_train, padding=True, truncation=True)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True)
Y_val_tokenized = tokenizer(y_val, padding=True, truncation=True)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [21]:
class CustomDataset(Dataset):
    def __init__(self, X_tokenized, Y_tokenized):
        self.input_ids = X_tokenized["input_ids"]
        self.attention_mask = X_tokenized["attention_mask"]
        self.labels = Y_tokenized["input_ids"]

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        input_ids = self.input_ids[idx]
        attention_mask = self.attention_mask[idx]
        labels = self.labels[idx]
        return torch.tensor(input_ids), torch.tensor(attention_mask), torch.tensor(labels)

In [22]:
batch_size = 32
epochs = 3

In [23]:
train_dataset = CustomDataset(X_train_tokenized, Y_train_tokenized)
val_dataset = CustomDataset(X_val_tokenized, Y_val_tokenized)

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=16, shuffle=True)

In [24]:
model = RobertaForCausalLM.from_pretrained("DDSC/roberta-base-danish")
device = 'cuda' if torch.cuda.is_available() else 'cpu'
torch.device(device)
model = model.to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
model.parameters()


If you want to use `RobertaLMHeadModel` as a standalone, add `is_decoder=True.`


<generator object Module.parameters at 0x7f3184eb4970>

In [25]:
def train(model, optimizer, train_dataloader, val_dataloader, num_epochs):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    print("Training on device:", device)

    model.to(device)
    model.train()

    for epoch in range(num_epochs):
        print("Epoch:", epoch + 1)
        train_loss = 0.0
        val_loss = 0.0

        # Training loop
        progress_bar = tqdm(train_dataloader, desc="Training")
        for batch in progress_bar:
            input_ids, attention_mask, labels = batch
            input_ids = input_ids.to(device)
            attention_mask = attention_mask.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss
            train_loss += loss.item()

            loss.backward()
            optimizer.step()

            progress_bar.set_postfix({"train_loss": loss.item()})

        # Validation loop
        model.eval()
        with torch.no_grad():
            progress_bar = tqdm(val_dataloader, desc="Validation")
            for batch in progress_bar:
                input_ids, attention_mask, labels = batch
                input_ids = input_ids.to(device)
                attention_mask = attention_mask.to(device)
                labels = labels.to(device)

                outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
                loss = outputs.loss
                val_loss += loss.item()

                progress_bar.set_postfix({"val_loss": loss.item()})

        # Print train and validation loss
        avg_train_loss = train_loss / len(train_dataloader)
        avg_val_loss = val_loss / len(val_dataloader)
        print("Train loss:", avg_train_loss)
        print("Validation loss:", avg_val_loss)

    print("Training completed!")


In [26]:
train(model, optimizer, train_dataloader, val_dataloader, epochs)

Training on device: cuda
Epoch: 1


Validation: 100%|██████████| 882/882 [00:35<00:00, 24.69it/s, val_loss=0.0122]


Train loss: 1.0493440072720868
Validation loss: 0.15334612928656122
Epoch: 2


Validation: 100%|██████████| 882/882 [00:35<00:00, 24.72it/s, val_loss=0.00224]


Train loss: 0.044239290373384935
Validation loss: 0.010061871196983315
Epoch: 3


Validation: 100%|██████████| 882/882 [00:36<00:00, 24.24it/s, val_loss=0.00034]

Train loss: 0.012856491777853203
Validation loss: 0.005279796224523968
Training completed!


In [27]:
torch.save(model, './tenseModel1')